In [6]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.misc
from matplotlib.pyplot import imshow
from PIL import Image

In [3]:
import tensorflow as tf

/Users/shashvatkedia/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
vgg = scipy.io.loadmat("")
vgg_layers = vgg["layers"]

In [ ]:
def weights(layer,expected_layer_name):
    W = vgg_layers[0][layer][0][0][0]
    b = vgg_layers[0][layer][0][0][1]
    layer_name = vgg_layers[0][layer][0][0][-2]
    assert layer_name == expected_layer_name
    return W,b

In [ ]:
def relu(cov2d_layer){
    return tf.nn.relu(conv2d_layer)
}

In [ ]:
def cond2d(prev_layer,layer,layer_name):
    W,b = weights(layer,layer_name)
    W = tf.constant(W)
    b = tf.constant(np.reshape(b,(b.size)))
    return tf.nn.conv2d(prev_layer,filter=W,strides=[1,1,1,1],padding='SAME') + b

In [ ]:
def avg_pool(layer):
    return tf.nn.avg_pool(layer,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [ ]:
graph = {}
graph['input'] = tf.Variable(np.zeros(1,600,800,3),dtype='float32')
graph['conv1_1'] = relu(conv2d(graph['input'],0,'conv1_1'))
graph['conv1_2'] = relu(conv2d(graph['conv1_1'],2,'conv1_2'))
graph['avgpool1'] = avg_pool(graph['conv1_2'])
graph['conv2_1'] = relu(conv2d(graph['avgpool1'],5,'conv2_1'))
graph['conv2_2'] = relu(conv2d(graph['conv2_1'],7,'conv2_2'))
graph['avgpool2'] = avg_pool(graph['conv2_2'])
graph['conv3_1'] = relu(conv2d(graph['avgpool2'],10,'conv3_1'))
graph['conv3_2'] = relu(conv2d(graph['conv3_1'],12,'conv3_2'))
graph['conv3_3'] = relu(conv2d(graph['conv3_2'],14,'conv3_3'))
graph['conv3_4'] = relu(conv2d(graph['conv3_3'],16,'conv3_4'))
graph['avgpool3'] = avg_pool(graph['conv3_4'])
graph['conv4_1'] = relu(conv2d(graph['avgpool3'],19,'conv4_1'))
graph['conv4_2'] = relu(conv2d(graph['conv4_1'],21,'conv4_2'))
hraph['conv4_3'] = relu(conv2d(graph['conv4_2'],23,'conv4_3'))
graph['conv4_4'] = relu(conv2d(graph['conv4_3'],25,'conv4_4'))
graph['avgpool4'] = avg_pool(graph['conv4_4'])
graph['conv5_1'] = relu(conv2d(graph['avgpool4'],28,'conv5_1'))
graph['conv5_2'] = relu(conv2d(graph['conv5_1'],30,'conv5_2'))
graph['conv5_3'] = relu(conv2d(graph['conv5_2'],32,'conv5_3'))
graph['conv5_4'] = relu(conv2d(graph['conv5_3'],34,;conv5_4))
graph['avgpool5'] = avg_pool(graph['conv5_4'])

In [ ]:
def content_loss(sess,model):
    def content_loss_util(p,x):
        N = p.shape[3]
        M = p.shape[1] * p.shape[2]
        return (1/(4*N*M)) * tf.reduce_sum(tf.pow(x-p,2))
    return content_loss_util(sess.run(model['conv4_2']),sess.run(model['conv4_2']))

In [ ]:
STYLE_LAYERS = [
    ('conv1_1',0.5),
    ('conv2_1',1.0),
    ('conv3_1',1.5),
    ('conv4_1',3.0),
    ('conv5_1',4.0),
]

In [ ]:
def style_loss(sess,model):
    def gram_matrix(F,N,M):
        Ft = tf.reshape(F,(N,M))
        return tf.matmul(tf.transpose(Ft),F)
    
    def style_loss_util(a,x):
        N = a.shape[3]
        M = a.shape[1] * a.shape[2]
        A = gram_matrix(a,N,M)
        G = gram_matrix(x,N,M)
        return (1/(4 * N**2 * M**2)) * tf.reduce_sum(tf.pow(G-A,2))
    E = [style_loss_util(sess.run(model[layer_name]),sess.run(model[layer_name])) for layer_name,_ in STYLE_LAYERS]
    W = [w for _,w in STYLE_LAYERS]
    return sum([W[l] * E[l] for l in range(len(STYLE_LAYERS))])

In [ ]:
def generate_noisy_image(content_image,noise_ratio=0.6):
    noisy_image = np.random.uniform(-20,20,(1,600,800,3)).astype('float32')
    return noisy_image * noise_ratio + content_image * (1 - noise_ratio)

In [ ]:
MEAN_VALUES = np.array([123.68,116.779,103.939])

In [ ]:
def load_image(path):
    image = scipy.misc.imread(path)
    image = np.reshape(image,((1,) + image.shape))
    image = image - MEAN_VALUES
    return image

In [ ]:
def save_image(path):
    image = image + MEAN_VALUES
    image = image[0]
    image = np.clip(image,0,255).astype('uint8')
    scipy.misc.imsave(path,image)

In [ ]:
sess = tf.InteractiveSession()
